In [26]:
%matplotlib widget
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets.widgets import HBox, VBox, Label, IntSlider, Output, IntRangeSlider

# Optional, for notebook only, not lab
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

cmap = 'jet'  #sets the colormap for the navigation image


def plot_polar(s):
    """
    Plot a 4D hyperspy dataset with the signal in polar coordinates
    Arguments:
    s: hyperspy signal of shape <X, Y, W| R>
    Notes:
    The theta-axis used is the signal axes' axis, which should be in degree units
    (check s.axes_manager[-1].axis)
    
    How to use:
    Must use the ipympl backend. %matplotlib widget
    Best viewed in jupyter lab or in the notebook with a stretched viewport, try:
    from IPython.core.display import display, HTML
    display(HTML("<style>.container { width:100% !important; }</style>"))
    """
    # Flip the signal and navigation with .T, then sum over the former signal dimension to produce a X Y W dataset. 
    # This becomes the figure we plot on the left.
    nav = s.T.sum() 

    # Set up the initial values for the rectangular region of interest marker
    # x coordinates
    value2index0 = nav.axes_manager[0].value2index
    index2value0 = nav.axes_manager[0].index2value

    low = nav.axes_manager[0].low_index
    high = nav.axes_manager[0].high_index
    left = index2value0(int(low + 1/4*(high-low)))
    right = index2value0(round(low + 3/4*(high-low)))

    # y coordinates
    value2index1 = nav.axes_manager[1].value2index
    index2value1 = nav.axes_manager[1].index2value
    
    low = nav.axes_manager[1].low_index
    high = nav.axes_manager[1].high_index
    top = index2value1(int(low + 1/4*(high-low)))
    bottom = index2value1(round(low + 3/4*(high-low)))

    # Since the navigator has three dimensions and we can only display two,
    # we add a slider to control the wavelength dimension
    wavelength_slider = IntSlider(value=0, min=0, max=nav.axes_manager[2].high_index)
    orientation_slider = IntSlider(value=0, min=0, max=nav.axes_manager[3].high_index)
    
####
    clim_slider = IntRangeSlider(value=[10000, 100000],
        min=0,
        max=250000,
        step=10000,
        description='clim:',
        disabled=False,
        continuous_update=True,
        orientation='vertical',
        readout=True,
        readout_format='d',
    )
####
    # create a dataset with only X and Y, slicing at the wavelength position
    nav_first_two = nav.isig[...,wavelength_slider.value, orientation_slider.value] # ... means "leave every axis until the end", equivalent of :,: here.
    
    # We use normal hyperspy plotting to display the first figure, because we want to use hyperspy ROIs.
    # Note that hs plotting really is matplotlib plotting with more on top.
    # Prevent the matplotlib figure from being created immediately by using an Output ipywidget. 
    # This is a separate area that we will display later
    nav_output = Output()
    with nav_output:
        nav_first_two.plot(cmap=cmap) #this generates the navigation image, other matplotlib parameters can be passed here.
    # get ahold of the image displayed in this figure
    # we will be changing the array displayed in `im` with `im.set_data()` later.   
    im = nav_first_two._plot.signal_plot.ax.images[0] 
    
    
    
    # Create the roi and place it on the left figure, then create labels that hold the name and position of the 
    # coordinates of the ROI and the wavelength index
    roi = hs.roi.RectangularROI(left, top, right, bottom)
    r = roi.interactive(nav_first_two)
    # The labels use f-strings, which are a convenient way to write out strings with objects in them. The :.2f operator means "leave two decimals and show as a float"
    xlabel = Label(value=f"{nav.axes_manager[0].name}: {roi.left:.2f} : {roi.right:.2f}")
    ylabel = Label(value=f"{nav.axes_manager[1].name}: {roi.top:.2f} : {roi.bottom:.2f}")
    wlabel = Label(value=f"{nav.axes_manager[2].name}: {nav.axes_manager[-1].index2value(wavelength_slider.value):.2f}")
    olabel = Label(value=f"{nav.axes_manager[3].name}: {nav.axes_manager[-1].index2value(orientation_slider.value):.2f}")
    # Plot right figure using an Output widget to prevent it being shown immediately
    sig_output = Output()
    with sig_output:
        fig = plt.figure()
    ax = plt.subplot(111, projection='polar')
    ax.set_xticks([0,np.pi/3,2*np.pi/3,np.pi,4*np.pi/3,5*np.pi/3])
    ax.tick_params(axis='both', labelsize=20)
    ax.set_rticks([600, 1200])
    ax.autoscale(enable=False)
    thetaaxis = s.axes_manager[-1].axis * np.pi / 180 # convert x axis to radians
    # Plot the polar plot. We return the line plotted into `lns` (which is a list of all lines)
    lns = ax.plot(
        thetaaxis,
        s.data[orientation_slider.value,wavelength_slider.value,
               value2index1(roi.top):value2index1(roi.bottom), 
               value2index0(roi.left):value2index0(roi.right)
              ].mean((0,1)))
    ln = lns[0] # lns only contains one line, which we will need later to update it

    def update(roi):
        "Update the polar plot and label values from a moving ROI"
        # Here we index the numpy array instead of the hyperspy .inav, because the former is quite a bit faster.
        # the numpy array is indexed in reverse order of hyperspy for some reason
        # The specific "roi" object name is imporant, as the `connect` method we use later expects it.
        
        # Update plot with set_ydata
        ydata = s.data[orientation_slider.value,wavelength_slider.value,
               value2index1(roi.top):value2index1(roi.bottom), 
               value2index0(roi.left):value2index0(roi.right)
              ].mean((0,1)) # 0 and 1 are the X and Y axes here, because we eliminate the wavelength axis by indexing it not using a range
        ln.set_ydata(ydata)
########ax.set_ylim(0, ydata.max()) # Polar plots look funny if we index with ydata.min() instead of 0.
        ax.set_ylim(0, 1200)
        
        
        xlabel.value=f"{nav.axes_manager[0].name}: {roi.left:.2f} : {roi.right:.2f}"
        ylabel.value=f"{nav.axes_manager[1].name}: {roi.top:.2f} : {roi.bottom:.2f}"
        wlabel.value=f"{nav.axes_manager[2].name}: {nav.axes_manager[-1].index2value(wavelength_slider.value):.2f}"
        olabel.value=f"{nav.axes_manager[3].name}: {nav.axes_manager[-1].index2value(orientation_slider.value):.2f}"
        
    def update_from_slider(change):
        "Update the polar plot, label values AND left hand plot by changing the wavelength slider"
        # Here we index the numpy array instead of the hyperspy .inav, because the former is quite a bit faster.
        # the numpy array is indexed in reverse order of hyperspy for some reason
        # The `change` object name is important, ipywidget `observe` methods expect it.
        # While I do use the change['new'] object here, I could equally use `wavelength_slider.value`
        wavelength_index = wavelength_slider.value
        orientation_index = orientation_slider.value
        # Update image with set_data
        im.set_data(s.data[orientation_index, wavelength_index].sum(-1)) # update the navigator, first index here is the wavelength, : or ... after is unnecessary.
        
        
########im.autoscale() # autoscale the nav image  
        im.set_clim(clim_slider.value)    #.  This sets the min and max of the color scale in the navigation image
    
    
        # Update plot with set_ydata
        ydata = s.data[orientation_index, wavelength_index,
               value2index1(roi.top):value2index1(roi.bottom), 
               value2index0(roi.left):value2index0(roi.right)
              ].mean((0,1)) # 0 and 1 are the X and Y axes here, because we eliminate the wavelength axis by indexing it not using a range
        ln.set_ydata(ydata)
########ax.set_ylim(0, ydata.max()) # Polar plots look funny if we index with ydata.min() instead of 0.
        ax.set_ylim(0, 1200)
        
        xlabel.value=f"{nav.axes_manager[0].name}: {roi.left:.2f} : {roi.right:.2f}"
        ylabel.value=f"{nav.axes_manager[1].name}: {roi.top:.2f} : {roi.bottom:.2f}"
        wlabel.value=f"{nav.axes_manager[2].name}: {nav.axes_manager[-1].index2value(wavelength_index):.2f}"
        olabel.value=f"{nav.axes_manager[3].name}: {nav.axes_manager[-1].index2value(orientation_index):.2f}"

    roi.events.changed.connect(update) # Call the `update(roi=roi)` function every time the ROI changes, with the latest values of the `roi` object
    wavelength_slider.observe(update_from_slider, names='value') # Call the `update_from_slider(change=change)` function every time the slider changes, 
    orientation_slider.observe(update_from_slider, names='value')
    clim_slider.observe(update_from_slider, names='value')
    # With ipympl enabled, the figure.canvas object is an ipywidget itself. 
    # That means can we can modify attributes like margin and other positioning. See the ipywidgets docs for more info
    navfig = nav_first_two._plot.signal_plot.figure.canvas 
    
    sigfig = fig.canvas

    navfig.layout.margin = "auto 0px auto 0px" # This centers the figure, I think. Can't actually remember.
    sigfig.layout.margin = "auto 0px auto 0px" # This centers the figure, I think. Can't actually remember.
    sigfig.header_visible = False # removes unnecssary "Figure 1" info on top
    navfig.header_visible = False
    display(HBox([nav_output, sig_output])) # Here we acutally display the figures, wrapped in a "Horizontal Box" widget so that they are next to each other
    display(HBox([Label('Wavelength:'), wavelength_slider,HBox([Label('Orientation'),orientation_slider]),clim_slider, VBox([xlabel, ylabel, wlabel, olabel])]))
    
# Use:

#s = hs.load(r"fixed_2020-04-06.hspy")

A quick theoretical overview: 

(1) $P_i (\omega_{SHG})$ = $\epsilon_0 \sum_{jk} \chi_{ijk}^{(2)} (\omega_{SHG},\omega_{inc},\omega_{inc})$ $E_j(\omega_{inc}) E_k(\omega_{inc})$

This polarization field describes the Second Harmonic Generation (SHG) emission of laser light. This can be derived classically from the nonlinear anharmonic oscillator equation, or from Maxwell's equations with equal validity. The polarization field is dominated by the second order susceptibility tensor $\chi^{2}$, which can be derived in various ways but chief among them is the density matrix formalism of quantum mechanics. In the interest of time we'll skip the derivation as it takes up nearly 11 pages in the relevant textbook<sup>1</sup>. The important thing to note here is that there is a spatial dependence in the material parameter $\chi^{2}$, and that the density matrix approach yields information about its behavior both close to electronic resonances and far away from them. 

In particular, SHG has been suggested as a way to probe so-called 'dark' states, excitonic resonances that are invisible to standard absorption/emission linear optical techniques. 

1: Nonlinear Optics, Robert Boyd, 3rd edition

In [6]:
s = hs.load(r"fixed_2020-04-06.hspy")

In [27]:
plot_polar(s)

In [19]:
im

NameError: name 'im' is not defined